# Взаимодействие с GMail через API

## Установка библиотек
Для установки библиотек выполнить следующий код
```
pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib
```

## Импорт библиотек

In [1]:
import os
import pickle
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from base64 import urlsafe_b64decode, urlsafe_b64encode
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from mimetypes import guess_type as guess_mime_type

## Указание сервера и почты
Указываем почтовый сервер и почту, к которой необходимо получить доступ.

In [ ]:
SCOPES = ['https://mail.google.com/']
our_email = 'mail@gmail.com'

## 1. Авторизация
#### Описание функции
Для авторизации необходимо: 
- Создать OAuth 2.0 Client;
- В настройках приложения предоставить доступ к нужной почте этому клиенту;
- Разрешить приложению использовать API GMail;
- Скачать JSON с данными подключения;
    
Это можно сделать по адресу: https://console.cloud.google.com/apis/credentials

In [130]:
def gmail_authenticate(path_credentials):
    ''' Функция для авторизации в GMail '''
    
    creds = None
    
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
            
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(path_credentials, SCOPES)
            creds = flow.run_local_server(port=0)
            
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)
            
    return build('gmail', 'v1', credentials=creds)

#### Пример
Подключаем при помощи сгенерированного ранее JSON с данными подключения.

In [139]:
path_credentials = 'credentials.json'
service = gmail_authenticate(path_credentials)

## 2. Поиск писем по строке
#### Описание функции
Поиск осуществляется по входящей строке. Описание метода доступно по ссылке: https://developers.google.com/gmail/api/reference/rest/v1/users.messages/list?hl=en

In [6]:
def search_messages(service, query, include_spam_trash):
    ''' Поиск писем '''
    
    result = service.users().messages().list(userId='me', includeSpamTrash=include_spam_trash).execute()
    messages = []
    
    if 'messages' in result:
        messages.extend(result['messages'])
    while 'nextPageToken' in result:
        page_token = result['nextPageToken']
        result = service.users().messages().list(userId='me',q=query, pageToken=page_token).execute()
        if 'messages' in result:
            messages.extend(result['messages'])
            
    return messages

#### Пример

In [ ]:
query = 'яндекс'
include_spam_trash = False

results = search_messages(service, query, include_spam_trash)

## 3. Чтение содержания писем
#### Описание функции
Получаем необходимую информацию о письмах. В рамках моей задачи вычитываем:
- id письма
- отправителя 
- получателя 
- тему
- дату 

In [7]:
def read_message(service, message_id):
    ''' Чтение писем '''
    
    msg = service.users().messages().get(userId='me', id=message_id['id'], format='full').execute()

    payload = msg['payload']
    headers = payload.get("headers")
    parts = payload.get("parts")
    folder_name = "email"
    
    _message_id = message_id['id']
    _from = ''
    _to = ''
    _subject = ''
    _date =  ''
    
    if headers:
        for header in headers:
            
            name = header.get("name")
            value = header.get("value")
            
            if name.lower() == 'from':
                _from = value
                
               
            if name.lower() == "to":
                _to = value
                
                
            if name.lower() == "subject":
                _subject = value
                
                
            if name.lower() == "date":
                _date = value
                
        
        return [_message_id, _from, _to, _subject, _date]

#### Пример
После чтения данных, для удобства дальнейшей работы формируем список списков и создаем **Pandas.DataFrame**

In [10]:
import pandas as pd

list_of_msg = []

for msg in results:
    msg_info = read_message(service, msg)
    list_of_msg.append(msg_info)
    
df = pd.DataFrame(list_of_msg, columns=['message_id','from', 'to', 'subject', 'date'])

## 4. Таргетированное удаление
#### Описание функции

В API GMail есть возможность таргетированного удаления писем по id письма. Описание метода доступно по ссылке: https://developers.google.com/gmail/api/reference/rest/v1/users.messages/delete?hl=en

In [129]:
def delete_message(service, message_id):
    ''' Таргетированное удаление письма '''
    
    service.users().messages().delete(
      userId='me',
      id= message_id
    ).execute()

#### Пример

In [ ]:
message_id = 'some_message_id'

delete_message(service, message_id)

## 5. Перенос в корзину

#### Описание функции

Перенос в корзину осуществляется по id письма. Описание метода доступно по ссылке: https://developers.google.com/gmail/api/reference/rest/v1/users.messages/trash?hl=en

In [137]:
def put_to_trash(service, message_id):
    ''' Перенос в корзину '''
    
    service.users().messages().trash(
      userId='me',
      id= message_id
    ).execute()  

#### Пример 

In [ ]:
for index, row in df[~df['domen'].isin(['@mail.ru', '@gmail.com'])].iterrows():
    put_to_trash(service, row['message_id'])